In [1]:
import sys 
sys.path.append('../')
import numpy as np
from dataset import LincsDataset
from torch_geometric.loader import DataLoader
import torch
from omegaconf import OmegaConf
from model_utils import get_params
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from datetime import datetime
from moler_ldm import LatentDiffusion

2023-05-05 22:14:55.875224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
batch_size = 1
NUM_WORKERS = 4
train_split1 = "train_0"
valid_split = "valid_0"

In [4]:
raw_moler_trace_dataset_parent_folder = "/data/ongh0068/guacamol/trace_dir"
output_pyg_trace_dataset_parent_folder = (
    "/data/ongh0068/l1000/already_batched"
)

In [8]:
config_file = 'config/ddim_aae_con.yml'
config = OmegaConf.load(config_file)
ldm_params = config['model']['params']
ldm_params

{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 50, 'timesteps': 200, 'first_stage_key': 'image', 'cond_stage_key': 'gene_expressions', 'image_size': 512, 'channels': 1, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 1, 'use_ema': False, 'parameterization': 'eps'}

In [9]:
layer_type = "FiLMConv"
model_architecture = 'aae'
gradient_clip_val = 1.0
max_lr = 1e-5
gen_step_drop_probability = 0.95
use_oclr_scheduler = True
using_cyclical_anneal = False
use_clamp_log_var = False

In [11]:
train_dataset = LincsDataset(
    root="/data/ongh0068",
    raw_moler_trace_dataset_parent_folder=raw_moler_trace_dataset_parent_folder,  # "/data/ongh0068/l1000/trace_playground",
    output_pyg_trace_dataset_parent_folder=output_pyg_trace_dataset_parent_folder,
    gene_exp_controls_file_path="/data/ongh0068/l1000/lincs/robust_normalized_controls.npz",
    gene_exp_tumour_file_path="/data/ongh0068/l1000/lincs/robust_normalized_tumors.npz",
    lincs_csv_file_path="/data/ongh0068/l1000/lincs/experiments_filtered.csv",
    split=train_split1,
    gen_step_drop_probability=gen_step_drop_probability,
)

Loading controls gene expression...
Loading tumour gene expression...
Loading csv...


In [14]:
train_dataset._gen_step_drop_probability

0.95